In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt


In [2]:
loc = "C:/Users/scvch/Documents/Industry_Practicum/Data/"

## Events Data

In [3]:
def read_data(loc):
    ## Reading in the Events data and changing EventTimestamp to appropriate date format
    Events = pd.read_csv(loc + "Events.txt",sep = "\t")
    Events['EventTimestamp'] = pd.to_datetime(Events['EventTimestamp']).dt.date
    #Events['ModifyDate'] = pd.to_datetime(Events['ModifyDate']).dt.date
    #Events['JoinedDate'] = pd.to_datetime(Events['JoinedDate']).dt.date

    # Creating EventTimestamp for 2021 and changing to appropriate date format
    Event_sep_2021 = Events[Events['EventTimestamp'] <= datetime.strptime('2021-09-01',"%Y-%m-%d").date()]
    #Events = pd.read_csv(loc + "Events.txt",sep = "\t",low_memory = False,skiprows = 1000000,nrows = 3000000)



    ## Reading in the Requests data and changing RequestDate to appropriate format
    Requests = pd.read_csv(loc + "Requests.txt",sep = "\t")
    Requests['RequestDate'] = pd.to_datetime(Requests['RequestDate']).dt.date

    ## Reading in the Request Resorts data
    Request_Resorts = pd.read_csv(loc + "Request Resorts.txt",sep = "\t")


    ## Reading in the request_offers data and changing OfferDate to appropriate format
    Request_offers = pd.read_csv(loc + "Request Offers.txt",sep = "\t")
    Request_offers['OfferDate'] = pd.to_datetime(Request_offers['OfferDate']).dt.date


    ## Reading in the Inventory data and using Exchange suppliedAs, Member purchaser base type, and converting depositDate to date date format
    Inventory = pd.read_csv(loc + "Inventory.txt",sep = "\t")
    Inventory = Inventory[Inventory['suppliedAs'] == 'Exchange']
    Inventory = Inventory[Inventory['purchaserbaseType'] == 'MEMBER']
    Inventory['depositDate'] = pd.to_datetime(Inventory['depositDate']).dt.date
    
    df_Resort_Details=pd.read_csv(loc + 'Resort Details.txt', delimiter="\t")
    #df_Resort_Details.head()
    df_Attributes=pd.read_csv(loc + 'Attributes.txt', delimiter="\t")
    #df_Attributes.head()
    df_Resort_Other_Attributes=pd.read_csv(loc + 'Resort Other Attributes.txt', delimiter="\t")
    df=df_Resort_Other_Attributes
    
    return Events,Requests,Request_Resorts,Request_offers,Inventory,df_Resort_Details,df_Attributes,df_Resort_Other_Attributes


In [4]:
Events,Requests,Request_Resorts,Request_offers,Inventory,df_Resort_Details,df_Attributes,df_Resort_Other_Attributes = read_data(loc) 

C:\Users\scvch\anaconda4\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## Request and search data


In [5]:
Accounts = pd.read_csv(loc + "Account.txt",sep = "\t")
Accounts['AccountStatus'].fillna('Active',inplace = True)


C:\Users\scvch\anaconda4\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Resort Data cleaning

In [6]:
# Obtain Unique ResourtID, AttrType and AttrID values
ls_resort_id = list(df_Resort_Other_Attributes['ResortID'].unique())
ls_resort_id
ls_attr_type = list(df_Resort_Other_Attributes['AttrType'].unique())
ls_attr_type
ls_attr_id = list(df_Resort_Other_Attributes['AttrID'].unique())
#ls_attr_id


# Create data final dictionary and put Resort ID and AttrType 1 dimensional vectors
data_final={}
data_final['Resort ID']=[]
data_final['Attr Type']=[]
for i in ls_attr_id:
    data_final[i]=[]
    
    
for resort in ls_resort_id:
    #print (resort)
    # where ResortID = resort and AttrType = type, appending them accordingly to those variables and creating a list with unique AttributeIDs
    for type in ls_attr_type:
        df_temp=df_Resort_Other_Attributes[(df_Resort_Other_Attributes['ResortID']==resort) & (df_Resort_Other_Attributes['AttrType']==type)]
        data_final['Attr Type'].append(type)
        data_final['Resort ID'].append(resort)
        ls_id=list(df_temp['AttrID'].unique())
        # Appending 1 to data_final vector and creating ls_id complement
        for j in ls_id:
            data_final[j].append(1)
        ls_id_complement=list(set(ls_attr_id)-set(ls_id))
        # Append 0 to data_final vector for every k in lis_id_complement
        for k in ls_id_complement:
            data_final[k].append(0) 

df_final=pd.DataFrame(data_final)
#df_final1=df_final.set_index(['Resort ID','Attr Type',df.groupby(['Resort_ID']).cumcount()+1]).unstack().sort_index(level=1,axis=1)
df_final1=df_final.set_index(['Resort ID','Attr Type']).unstack().sort_index(level=1)
                             
#df_out = df.set_index(['IndividualID','DayID',df.groupby(['IndividualID','DayID']).cumcount()+1]).unstack().sort_index(level=1, axis=1)
#df_final.columns = df_final.columns.map('{0[0]}_{0[1]}'.format)
#df_final1.reset_index()
col=list(df_final1.columns)
final_col=[str(i) for i in col]
final_col
df_final1.columns = final_col



In [7]:
df_final1

,"(60, 9)","(60, 11)","(60, 13)","(46, 9)","(46, 11)","(46, 13)","(62, 9)","(62, 11)","(62, 13)","(14, 9)",...,"(4049, 13)","(831, 9)","(831, 11)","(831, 13)","(1195, 9)","(1195, 11)","(1195, 13)","(4061, 9)","(4061, 11)","(4061, 13)"
Resort ID,,,,,,,,,,,,,,,,,,,,,
$99BVOU,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10026,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10161,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZJHH,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
ZURICH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZWCARB,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Create df_final as dataframe
#df_final=pd.DataFrame(data_final)
resort_dt = df_final1

# Compute cosine similarity wth resort tables
cosine_sim = linear_kernel(resort_dt, resort_dt)
# show cosine similarity shape

# reset Index for Resort table
resort_new = resort_dt.reset_index()

#Construct a reverse map of indices and movie titles
indices = pd.Series(resort_new.index,index = resort_new['Resort ID']).drop_duplicates()


In [12]:
#create recommendation of similar resourts via movie titles
    
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title

    if str(title) in resort_new['Resort ID'].unique() : 
        idx = indices[title]

        # Get the pairwsie similarity scores of all resorts with the resort
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Sort the resorts based on the similarity scores
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Get the resorts of the 5 most similar resorts
        sim_scores = sim_scores[1:6]
        
        # Get the resort indices
        resort_indices = [i[0] for i in sim_scores]
        return list(resort_new['Resort ID'].iloc[resort_indices])
    else :
        return None
    
    

### Factors 

1. Number of searches (1,0.5,0.25,0.125)
2. Number of Requests
3. Number of Bookings
4. Number of cancellations


## Bookings interaction

In [13]:

# Using those Events where something was purchased, create Bookings_new by inner joining Bookings with Inventory Tables
Bookings = Events[(Events['eventName'] == 'Purchased Something')]
Bookings_new = pd.merge(Bookings,Inventory,left_on = 'DepositID',right_on = 'depositId',how = 'inner')
Bookings_new = Bookings_new[['AccountID','EventTimestamp','resortId']]
#Bookings_new.groupby("AccountID")["EventTimestamp"].rank(ascending=0,method='dense')

# Create the days_since variable as number of days since booking happened
Bookings_new['days_since'] = datetime.strptime('2022-01-01',"%Y-%m-%d").date() - Bookings_new['EventTimestamp']
Bookings_new['days_since'] = Bookings_new['days_since'].dt.days

# Rank new Bookings by days since booking happened
Bookings_new['rank'] = Bookings_new.groupby("AccountID")["days_since"].rank(ascending=1,method='dense')
Bookings_new["Bookings"] = 1/Bookings_new["rank"]
Bookings_new = Bookings_new[['AccountID','resortId','Bookings']].groupby(['AccountID','resortId']).sum().reset_index()


## Request Interaction

In [14]:

# Using those Events where member request was palced, create Requests_new by inner joining Requests with request_resorts Tables
Requests = Events[(Events['eventName'] == 'Member Request Placed')]
Request_Resorts.rename(columns = {'ResortID':'resortId'}, inplace = True)
Requests_new = pd.merge(Requests,Request_Resorts,left_on = 'RequestID',right_on = 'RequestID',how = 'inner')
Requests_new = Requests_new[['AccountID','EventTimestamp','resortId']]
#Bookings_new.groupby("AccountID")["EventTimestamp"].rank(ascending=0,method='dense')

# Create the days_since variable as number of days since requests happened
Requests_new['days_since'] = datetime.strptime('2022-01-01',"%Y-%m-%d").date() - Requests_new['EventTimestamp']
Requests_new['days_since'] = Requests_new['days_since'].dt.days

# Rank new Requests by days since request was made
Requests_new['rank'] = Requests_new.groupby("AccountID")["days_since"].rank(ascending=1,method='dense')
Requests_new["Requests"] = 1/Requests_new["rank"]
Requests_new = Requests_new[['AccountID','resortId','Requests']].groupby(['AccountID','resortId']).sum().reset_index()

## Search Interaction

In [15]:
# Obtain by member search availability, innher join Search and Request_Resorts tables
Search = Events[(Events['eventName'] == 'Member Search Availability')]
Search_new = pd.merge(Search,Request_Resorts,left_on = 'RequestID',right_on = 'RequestID',how = 'inner')
Search_new = Search_new[['AccountID','EventTimestamp','resortId']]
#Bookings_new.groupby("AccountID")["EventTimestamp"].rank(ascending=0,method='dense')

# Create the days_since variable as number of days since member availability was searched
Search_new['days_since'] = datetime.strptime('2022-01-01',"%Y-%m-%d").date() - Search_new['EventTimestamp']
Search_new['days_since'] = Search_new['days_since'].dt.days
Search_new['rank'] = Search_new.groupby("AccountID")["days_since"].rank(ascending=1,method='dense')

# Rank new searches by days since search was made (recency of search)
Search_new["searches"] = 1/Search_new["rank"]
Search_new = Search_new[['AccountID','resortId','searches']].groupby(['AccountID','resortId']).sum().reset_index()
Search_new.rename(columns = {'Requests':'Searches'}, inplace = True)

## Cancellation requests


In [16]:

# Obtained by member request Placed, inner join Request_offers and Request_Resorts tables
cancels_data = pd.merge(Request_offers,Request_Resorts,on = 'RequestID')
cancels_data = cancels_data[cancels_data['OfferStatus'] == 'CANCELLED']
Requests = Events[(Events['eventName'] == 'Member Request Placed')]

# Further inner join Requests and Cancels_data tables with AccountID, EventTimestamp and resortId variables
cancels_new = pd.merge(Requests,cancels_data,left_on = 'RequestID',right_on = 'RequestID',how = 'inner')
cancels_new = cancels_new[['AccountID','EventTimestamp','resortId']]
#Bookings_new.groupby("AccountID")["EventTimestamp"].rank(ascending=0,method='dense')

# Create the days_since variable as number of days since a request was cancelled, subtracted from when a request was placed
cancels_new['days_since'] = datetime.strptime('2022-10-01',"%Y-%m-%d").date() - cancels_new['EventTimestamp']
cancels_new['days_since'] = cancels_new['days_since'].dt.days

# Rank by recency of request cancelled
cancels_new['rank'] = cancels_new.groupby("AccountID")["days_since"].rank(ascending=1,method='dense')
cancels_new["Cancels"] = -1/cancels_new["rank"]
cancels_new = cancels_new[['AccountID','resortId','Cancels']].groupby(['AccountID','resortId']).sum().reset_index()


### Making all the accounts interactions

In [18]:

# Reading Account table, and converting JoinedDate and ModifyDate to date formats
Accounts = pd.read_csv(loc + "Account.txt",sep = "\t")
Accounts['AccountStatus'].fillna('Active',inplace = True)
Accounts['JoinedDate'] = pd.to_datetime(Accounts['JoinedDate']).dt.date
Accounts['ModifyDate'] = pd.to_datetime(Accounts['ModifyDate']).dt.date

# Reading Resort Details, chainging updateDate column to date format and dropping 'territoryId','territory','AccountId','mapDestOfficeKey','busDevUserId','busDevUser','daeOptions','excludeFromAvailability' columns
resort = pd.read_csv(loc + "Resort Details.txt",sep = "\t")
resort = resort.drop(columns = ['territoryId','territory','AccountId','mapDestOfficeKey','busDevUserId','busDevUser','daeOptions','excludeFromAvailability'])
resort['updatedDate'] = pd.to_datetime(resort['updatedDate']).dt.date


C:\Users\scvch\anaconda4\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
#Setting key as 0 for Accounts and Resort
Accounts = Accounts[['accountid']]
Accounts['key'] = 0
resort = resort[['resortId']]
resort['key'] = 0


# Outer join merging new Bookings with new requests,new searches and later new cancellations tables to crete all interactions
dt = Bookings_new.merge(Requests_new,how = 'outer',on = ['AccountID','resortId']).merge(Search_new,how = 'outer',on = ['AccountID','resortId'])

dt = dt.merge(cancels_new,how = 'outer',on = ['AccountID','resortId'])
dt.fillna(0,inplace = True)

In [20]:
dt

,AccountID,resortId,Bookings,Requests,searches,Cancels
0,1303,CEDL,1.000000,0.000000,0.5,0.0
1,1303,SHEW,5.000000,1.000000,2.0,0.0
2,1348,CAPW,0.125000,0.333333,0.0,0.0
3,1348,LAKE,0.166667,0.333333,0.0,0.0
4,1348,LAKS,1.500000,0.000000,3.0,0.0
...,...,...,...,...,...,...
922542,7126282,SILS,0.000000,0.000000,4.0,0.0
922543,7126292,MISTY,0.000000,0.000000,1.0,0.0
922544,7126292,MPKLOP,0.000000,0.000000,1.0,0.0
922545,7126292,MSSA,0.000000,0.000000,7.0,0.0


In [21]:
# creating interaction data with more weightage to bookings and cancellations
dt['interaction'] = 5*dt['Bookings'] + dt['Requests'] + dt['searches'] + 5*dt['Cancels']

# creating resort_ranking data by grouping by AccountID and ranking interaction
dt['resort_ranking'] = dt.groupby(['AccountID'])['interaction'].rank(ascending=0,method='dense')

# creating Content recommendations for 1 or 2
dt_content_recommendations = dt[(dt['resort_ranking']==1) | (dt['resort_ranking']==2) ]


# apply get_recomemndations variable to resortId
dt_content_recommendations['recommendations'] = dt_content_recommendations['resortId'].apply(get_recommendations)



# Add recommendations table and predicted_accounts table to existing master attribute dataset for Contet_recommendations and predicted_accounts
dt_content_recommendations = pd.read_csv(loc + 'recommendations.csv' )
predicted_accounts = pd.read_csv(loc + 'predicted_accounts.csv' )


# Created predicted_accounts table
predicted_accounts.columns = ['index','account_id']
predicted_accounts = predicted_accounts[['account_id']] 
#Master_data =  pd.read_csv(loc + 'Master_data.csv' )


# Set up train start data as Jan 2022 and a season length of 3 months for training data
train_start_date = '2022-01-01'
season_length = 3




<ipython-input-21-c903ddafe907>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt_content_recommendations['recommendations'] = dt_content_recommendations['resortId'].apply(get_recommendations)


In [22]:

# Created s date variable, offsetting for 3 months
s = datetime.strptime(train_start_date,"%Y-%m-%d").date() - pd.DateOffset(months = season_length)
s = s.date()

## Reading in the account type data and using only Member status
Account_type = pd.read_csv(loc + "Account Type.txt",sep = "\t")
Account_type = Account_type[Account_type['BaseType'] == 'MEMBER']

## Reading in the accounts data and using only Active status
Accounts = pd.read_csv(loc + "Account.txt",sep = "\t")
Accounts['AccountStatus'].fillna('Active',inplace = True)

## Converting JoinedDate and ModifyDate columns to appropriate format
Accounts['JoinedDate'] = pd.to_datetime(Accounts['JoinedDate']).dt.date
Accounts['ModifyDate'] = pd.to_datetime(Accounts['ModifyDate']).dt.date
Accounts = pd.merge(Accounts,Account_type,on = 'AccountType',how = 'inner')

# Accounted for Time period for JoinedDate, and created no. of Membership days
Accounts = Accounts[Accounts['JoinedDate'] < s] 
Accounts['Membership days'] = s - Accounts['ModifyDate']  
Accounts['Membership days'] = Accounts['Membership days'].dt.days
Accounts = Accounts[Accounts['AccountStatus'] == 'Active']

# Inner joined Accounts with Account_type and Account and Account_Members and Events to get Account and Event Members tables
Account_Members = pd.merge(Accounts,Account_type,how = 'inner',left_on = 'AccountType',right_on = 'AccountType')[["accountid"]]
Event_Members = pd.merge(Account_Members,Events,how = 'inner',left_on = 'accountid',right_on = 'AccountID')

## Calculating Events for season 9 in the 3 month period for event where something was purchased (dropping duplicates)
Events_s9 = Event_Members[(Event_Members['EventTimestamp'] > s) & (Event_Members['EventTimestamp'] < s + pd.DateOffset(months=3)) ]
Events_s9 = Events_s9[Events_s9['eventName']=='Purchased Something'].drop_duplicates()
Events_s9['response'] = 1 

# dropping duplicates from account id for all_accounts, and accountid and response for response variable
all_accounts = Event_Members[['accountid']].drop_duplicates()
response = Events_s9[['accountid','response']].drop_duplicates()

# Setting up response table by merging all_accounts and response, filling null values with 0, and making into integer type
response = pd.merge(all_accounts,response,how = 'left',left_on = 'accountid',right_on = 'accountid')
response['response'].fillna(0,inplace = True)
response['response']= response['response'].astype(int)


# inner join Accounts and response tables
Accounts = pd.merge(Accounts,response,on = 'accountid',how = 'inner') 




C:\Users\scvch\anaconda4\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [23]:
# Total number where there are responses
sum(response['response']==1)

4696

In [24]:
# create final accounts as predicted account - current accounts where there is response
accounts = response[response['response'] == 1]['accountid']
final_accounts = set(predicted_accounts['account_id']) - set(accounts)
final_accounts = pd.DataFrame(final_accounts)
final_accounts.columns = ['accountid']
#final_accounts
#dt_content_recommendations

# Inner join final_accounts onto content recommendations table
dt_content_recommendations = pd.merge(final_accounts,dt_content_recommendations, how = 'inner', left_on = 'accountid',right_on = 'AccountID')

KeyError: 'AccountID'

In [ ]:

# Obtained number of times a resort was booked and merged final_accounts to it
resorts_booked = Events[Events['eventName']== 'Purchased Something']

resorts_booked = pd.merge(final_accounts,resorts_booked,how = 'inner',left_on = 'accountid', right_on = 'AccountID')

# Added Inventory data to current dataset where members were given exchange. 'depositDate' format was also changed
Inventory = pd.read_csv(loc + "Inventory.txt",sep = "\t")
Inventory = Inventory[Inventory['suppliedAs'] == 'Exchange']
Inventory = Inventory[Inventory['purchaserbaseType'] == 'MEMBER']
Inventory['depositDate'] = pd.to_datetime(Inventory['depositDate']).dt.date

# only kept AccountID and DepositID in resorts_booked, and changed DepositID to float type
resorts_booked = resorts_booked[['AccountID','DepositID']]
resorts_booked['DepositID'] = resorts_booked['DepositID'].astype('float')

# only kept resortId and DepositId in resorts_booked, and changed DepositId to float type
Inventory = Inventory[['depositId','resortId']]
Inventory['depositId'] = Inventory['depositId'].astype('float')

#Inner joined Inventory and resorts_booked
resorts_booked = pd.merge(Inventory,resorts_booked,left_on = 'depositId',right_on = 'DepositID' , how = 'inner')

resorts_booked

# Numbered resport_booked's idx variable by AccountID, pivoted and replaced null values
resorts_booked['idx'] = resorts_booked.groupby('AccountID').cumcount()
resorts_booked = resorts_booked.pivot(index = 'AccountID',columns='idx')[['resortId']]
resorts_booked = resorts_booked.fillna('')

# Started booked_resorts, a 1 dimensional vector
booked_resorts = []

freq = int(len(resorts_booked.values.flatten().tolist())/len(resorts_booked))

# rearranging resorts booked values in the table
for i in range(0,len(resorts_booked.values.flatten().tolist()),freq ):    
    booked_resorts.append(resorts_booked.values.flatten().tolist()[i:i+freq])
    
# Resetting index for resorts_booked
resorts_booked['booked_resorts'] = booked_resorts
resorts_booked = resorts_booked['booked_resorts'].reset_index()

#dt_content_recommendations['recommendations']

In [44]:
import string
str_list = lambda x : x.translate(str.maketrans('', '', string.punctuation)).split(' ')

# changing recommendations column to string, and apply str_list
dt_content_recommendations['recommendations'] = dt_content_recommendations['recommendations'].astype('str')
dt_content_recommendations['recommendations'] = dt_content_recommendations['recommendations'].apply(str_list)

# inner joining resorts_booked to our content recommendations data
dt_content_recommendations = pd.merge(resorts_booked,dt_content_recommendations,how = 'inner',on = 'AccountID')


def removing_past_resorts(x,y):
    return set(x)-set(y)

# looping such that we remove past resorts from our recommendations
dt_content_recommendations['new_recommendations'] = dt_content_recommendations.apply(lambda x: removing_past_resorts(x['recommendations'], x['booked_resorts']), axis=1)
dt_content_recommendations = dt_content_recommendations[['AccountID','new_recommendations']]
dt_content_recommendations['idx'] = dt_content_recommendations.groupby('AccountID').cumcount()
dt_content_recommendations = dt_content_recommendations.pivot(index = 'AccountID',columns='idx')[['new_recommendations']]
dt_content_recommendations = dt_content_recommendations.iloc[:,0:2].reset_index()
dt_content_recommendations.columns = ['Account_id','Set1_recommendation','Set2_recommendation']
    

In [45]:
dt_content_recommendations

,Account_id,Set1_recommendation,Set2_recommendation
0,26018,"{IA, R3735, BCOM, R0653, 3936}","{IA, CASB, R3683, R2826, SUNP}"
1,26030,"{IA, LAKS, BUSS, BELL, MANS}",NaN
2,26098,"{TWVC, IA, CABA, R3683, R2084}",NaN
3,26165,"{IA, LAKS, BUSS, BELL, MANS}",NaN
4,26202,"{EMET, IA, NEPC, SHEWR, CUMB}","{IA, LAKS, BUSS, BELL, MANS}"
...,...,...,...
13631,7117863,"{R2989, EW8677, R3574, R3683, R2414}","{EW5623, EW7053, EW5856, EW3880, EW4445}"
13632,7119649,"{IA, R3668, MOOR, R100142, MOOR3}","{EMET, BELL, IA, MOOR3}"
13633,7120107,"{EMET, IA, NEPC, R3683, NEPCSPA}","{EMET, R2395, BFRC, R2734, MGWC}"
13634,7121658,"{IA, MPNGWE, EW725, DRCASP, R3683}","{R0508, R0899, R0090, MIO, HILD}"


In [46]:
# create content_recommendations table
dt_content_recommendations.to_csv(loc + 'recommendations.csv')

In [ ]:
# call all functions

